In [2]:
using Random
using Random123
using Distributions
using Plots
using StatsPlots
using StatsBase
using Statistics
using DataFrames
using Chain
using Dates
include("Utils.jl")

generate_plots

In [3]:
# Initializing different prngs so that different replicates can be run
rng_default = Xoshiro(100)
Random.seed!(rng_default, 100)

# Seeding a rng for additive effects
additive_seed = 100 # changing this changes the topography of the landscape, depending on ruggedness
rng_additive = Xoshiro(additive_seed)
#Random.seed!(rng_additive, additive_seed)

# initializing a rng for genotype generation
genotype_seed = 100 # changing this changes the initial genotype generated
rng_init_genotype = Xoshiro(genotype_seed)
#Random.seed!(rng_init_genotype, genotype_seed)

# and a separate one for the initial genome
genome_seed = 100 # changing this changes the initial genome generated
rng_init_genome = Xoshiro(genome_seed)
#Random.seed!(rng_init_genome, genome_seed)

# as well as which mutants are created each generation (modeling different "decisions")
mutation_seed = 100 # changing this should change which decisions are made from some starting point
rng_mutation = Xoshiro(mutation_seed)
#Random.seed!(rng_mutations, mutation_seed)

# Initializing our relevant arguments
loci = 20 # since the bitwidth of the genome is 128, we can't actually model any arbitrary maximum genome bitsize. what we can do instead is enforce a maximum genome size (effectively, the bitwidth within a UInt128). 
init_active_loci = 3 # how many loci, out of the total number of loci in the landscape defined above, are unlocked at the start?
max_init_genotype_bits = 3 # makes the initial genotype reproducible between resticted and unlocked landscapes of the same size. max size is the number of init active loci
# what the above lets us model is how the restrictedness of the landscape at initialization (init_active_loci) determines what peaks are reached. max_init_genotype_bits allows
# us to control for the initial genotype when we change the number of unlocked loci at the start, otherwise we would just generate a different starting genotype (potentially one that wasn't available on the restricted landscape)
additive_effects = generate_additive_effects(128)#zeros(128) # we will always need to generate the full range of additive effects (128 bit)
total_population = 5000
σ_epi::Float64 = 0 # standard deviation of the epistatic effects
μ = (total_population^-1)/10 # Mutation rate of the genotypes with some genome. Claudia says than Nμ = 1 is a weird parameter regime, so we adjust it a bit lower
M = μ#*10^-1 # rate of genome evolution. expansion/streamlining events happen an order of magnitude less, on average, than mutations
simulation_length = 200000

2000

In [4]:
df_genotypes = simulate(loci, init_active_loci, max_init_genotype_bits, total_population, σ_epi, μ, M, simulation_length)
#generate_plots(df_genotypes)
sweeps_test = process_data(df_genotypes)[7] # assigns only the data from the selective sweep dataframe
sum(digits(sweeps_test[!,:Genome][end], base = 2)) # genome size at end of simulation

Iteration: 2000

Row,Step,Genotype,Genome,Pop
,Int64,UInt128,UInt128,Float64
1,0,0,4116,5000.0
2,1,4,4116,2.0
3,1,0,135188,2.0
4,1,0,4112,1.0
5,1,0,4116,4995.0
6,2,4,4116,2.0
7,2,0,135188,2.0
8,2,0,4112,1.0
9,2,0,4116,4995.0


9